# SQL Basics Assignment Questions | Maven Movies Database Queries


### Q1: Create a table called employees with the following structure
- `emp_id` (integer, should not be NULL and should be a primary key)  
- `emp_name` (text, should not be NULL)  
- `age` (integer, should have a check constraint to ensure the age is at least 18)  
- `email` (text, should be unique for each employee)  
- `salary` (decimal, with a default value of 30,000)  

Write the SQL query to create the above table with all constraints.

**Answer:** (see code cell below)

In [ ]:
import sqlite3

# Connect to an in-memory SQLite database (or create a file-based DB)
conn = sqlite3.connect('assignment.db')
cursor = conn.cursor()

# Execute the SQL to create the table
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER NOT NULL PRIMARY KEY,
    emp_name TEXT NOT NULL,
    age INTEGER CHECK(age >= 18),
    email TEXT UNIQUE,
    salary REAL DEFAULT 30000
)
""")

# Commit the changes
conn.commit()

# Verify the table was created
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:", tables)

Tables in database: [('employees',)]


### Q2. Explain the purpose of constraints and how they help maintain data integrity in a database. Provide examples of common types of constraints.

**Answer:**  
Constraints are rules applied to columns in a database table to enforce data integrity and ensure the accuracy, consistency, and validity of data. They prevent invalid or inconsistent data from being entered into the database.

**Common types of constraints:**

1. **PRIMARY KEY** – Ensures each row in a table is unique and not NULL.  
   *Example:* `emp_id INT PRIMARY KEY`  

2. **FOREIGN KEY** – Ensures that a value in one table matches a value in another table, maintaining referential integrity.  
   *Example:* `department_id INT, FOREIGN KEY (department_id) REFERENCES departments(dept_id)`  

3. **UNIQUE** – Ensures all values in a column are distinct.  
   *Example:* `email TEXT UNIQUE`  

4. **NOT NULL** – Ensures a column cannot have NULL values.  
   *Example:* `emp_name TEXT NOT NULL`  

5. **CHECK** – Ensures that values in a column satisfy a specific condition.  
   *Example:* `age INT CHECK (age >= 18)`  

6. **DEFAULT** – Provides a default value if none is specified.  
   *Example:* `salary DECIMAL(10,2) DEFAULT 30000`  

**Purpose:**  
- Prevents invalid or inconsistent data entries.  
- Maintains accuracy, reliability, and integrity of the database.  
- Helps enforce business rules at the database level rather than application code.

### Q3. Why would you apply the NOT NULL constraint to a column? Can a primary key contain NULL values? Justify your answer.

**Answer:**  
- **Purpose of NOT NULL constraint:**  
  The NOT NULL constraint ensures that a column cannot have NULL (empty) values. It is used to enforce that certain data must always be present in a table, which is important for critical fields like `emp_id`, `emp_name`, or `email`. This helps maintain data integrity by preventing missing or incomplete information.

- **Primary Key and NULL values:**  
  A primary key **cannot contain NULL values**.  
  **Justification:**  
  - The primary key uniquely identifies each row in a table.  
  - If a primary key allowed NULL, it would violate uniqueness because NULL is considered "unknown" and cannot reliably identify a row.  
  - Therefore, all primary key columns are implicitly NOT NULL and unique.

### Q4. Explain the steps and SQL commands used to add or remove constraints on an existing table. Provide an example for both adding and removing a constraint.

**Answer:**  
Constraints can be added to or removed from an existing table using the `ALTER TABLE` command.  

**Steps to add a constraint:**
1. Use `ALTER TABLE table_name ADD CONSTRAINT constraint_name constraint_type(column_name)`.
2. This will enforce the new rule on the specified column.

**Steps to remove a constraint:**
1. Use `ALTER TABLE table_name DROP CONSTRAINT constraint_name`.
2. This will remove the specified rule from the column.

**Examples:**

- **Add a constraint:** Ensure `age` is at least 18 in the `employees` table.  
- **Remove a constraint:** Remove the age check constraint if it is no longer needed.



In [ ]:
import sqlite3

# Connect to SQLite (or create database in memory)
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create employees table
cur.execute('''
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    emp_name TEXT NOT NULL,
    age INTEGER,
    email TEXT UNIQUE,
    salary REAL DEFAULT 30000
)
''')

# Add a CHECK constraint in SQLite requires creating a new table, example:
# SQLite does not support ALTER TABLE ADD CHECK directly, so normally done at table creation
# This is a known SQLite limitation

conn.commit()
conn.close()


### Q5. Explain the consequences of attempting to insert, update, or delete data in a way that violates constraints. Provide an example of an error message that might occur when violating a constraint.

**Answer:**  
Constraints in a database enforce rules to maintain data integrity. If an operation violates a constraint, the database **will reject the operation** and raise an error. This prevents invalid or inconsistent data from being stored.

**Consequences of violating constraints:**
1. **INSERT violation:** Attempting to insert a row that breaks a constraint (e.g., duplicate primary key, NULL in a NOT NULL column) will fail.  
2. **UPDATE violation:** Modifying a row in a way that violates a constraint (e.g., changing an email to a value that already exists in a UNIQUE column) will fail.  
3. **DELETE violation:** Deleting a row referenced by a foreign key in another table will fail if `ON DELETE RESTRICT` is set.

**Example of an error message:**
- Inserting a duplicate primary key:  

### Q6. You created a `products` table without constraints as follows:

```sql
CREATE TABLE products (
    product_id INT,
    product_name VARCHAR(50),
    price DECIMAL(10, 2)
);

Now, you realize that:  
- The `product_id` should be a primary key  
- The `price` should have a default value of 50.00  

**Answer:** (see code cell below)

In [ ]:
import sqlite3

# Connect to SQLite in memory (or a file)
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create the products table without constraints
cur.execute('''
CREATE TABLE products (
    product_id INT,
    product_name VARCHAR(50),
    price DECIMAL(10, 2)
)
''')

# SQLite does not support ALTER COLUMN to add DEFAULT or CHECK easily
# To add constraints in SQLite, you generally recreate the table with constraints
cur.execute('''
CREATE TABLE products_new (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50),
    price DECIMAL(10,2) DEFAULT 50.00
)
''')

# Copy data from old table if needed
cur.execute('INSERT INTO products_new (product_id, product_name, price) SELECT * FROM products')

# Drop old table and rename new table
cur.execute('DROP TABLE products')
cur.execute('ALTER TABLE products_new RENAME TO products')

conn.commit()
conn.close()


### Q7. You have two tables: Students and Classes.
Write a query to fetch the student_name and class_name for each student using an INNER JOIN.

**Answer:**  
We use INNER JOIN to combine the Students and Classes tables based on class_id.


In [ ]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create tables
cur.execute('''
CREATE TABLE Students (
    student_id INT,
    student_name TEXT,
    class_id INT
)
''')

cur.execute('''
CREATE TABLE Classes (
    class_id INT,
    class_name TEXT
)
''')

# Insert data
cur.executemany('INSERT INTO Students VALUES (?, ?, ?)', [(1, 'Alice', 101),
                                                         (2, 'Bob', 102),
                                                         (3, 'Charlie', 101)])

cur.executemany('INSERT INTO Classes VALUES (?, ?)', [(101, 'Math'),
                                                      (102, 'Science'),
                                                      (103, 'History')])

# Run INNER JOIN query
cur.execute('''
SELECT s.student_name, c.class_name
FROM Students s
INNER JOIN Classes c
ON s.class_id = c.class_id
''')

# Fetch and display results
for row in cur.fetchall():
    print(row)

conn.close()


('Alice', 'Math')
('Bob', 'Science')
('Charlie', 'Math')


### Q8. Write a query that shows all order_id, customer_name, and product_name, ensuring that all products are listed even if they are not associated with an order.
(Hint: use INNER JOIN and LEFT JOIN)


In [ ]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create tables
cur.execute('CREATE TABLE Orders (order_id INT, order_date TEXT, customer_id INT)')
cur.execute('CREATE TABLE Customers (customer_id INT, customer_name TEXT)')
cur.execute('CREATE TABLE Products (product_id INT, product_name TEXT)')
cur.execute('CREATE TABLE Order_Products (order_id INT, product_id INT)')  # Linking table

# Insert data
cur.executemany('INSERT INTO Orders VALUES (?, ?, ?)', [(1, '2024-01-01', 101),
                                                        (2, '2024-01-03', 102)])

cur.executemany('INSERT INTO Customers VALUES (?, ?)', [(101, 'Alice'),
                                                        (102, 'Bob')])

cur.executemany('INSERT INTO Products VALUES (?, ?)', [(1, 'Laptop'),
                                                       (2, 'Phone')])

cur.executemany('INSERT INTO Order_Products VALUES (?, ?)', [(1, 1)])  # Only one product linked

# Query using LEFT JOIN to include all products
cur.execute('''
SELECT o.order_id, c.customer_name, p.product_name
FROM Products p
LEFT JOIN Order_Products op ON p.product_id = op.product_id
LEFT JOIN Orders o ON op.order_id = o.order_id
LEFT JOIN Customers c ON o.customer_id = c.customer_id
''')

# Fetch and display results
for row in cur.fetchall():
    print(row)

conn.close()


(1, 'Alice', 'Laptop')
(None, None, 'Phone')


### Q9. Write a query to find the total sales amount for each product using an INNER JOIN and the SUM() function.


In [ ]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create tables
cur.execute('CREATE TABLE Sales (sale_id INT, product_id INT, amount INT)')
cur.execute('CREATE TABLE Products (product_id INT, product_name TEXT)')

# Insert data
cur.executemany('INSERT INTO Sales VALUES (?, ?, ?)', [
    (1, 101, 500),
    (2, 102, 300),
    (3, 101, 700)
])

cur.executemany('INSERT INTO Products VALUES (?, ?)', [
    (101, 'Laptop'),
    (102, 'Phone')
])

# Query: total sales per product
cur.execute('''
SELECT p.product_name, SUM(s.amount) AS total_sales
FROM Sales s
INNER JOIN Products p ON s.product_id = p.product_id
GROUP BY p.product_name
''')

# Fetch and display results
for row in cur.fetchall():
    print(row)

conn.close()


('Laptop', 1200)
('Phone', 300)


### Q10. Using the Orders, Customers, and Order_Details tables, write a query to display the order_id, customer_name, and the total quantity of products ordered by each customer using an INNER JOIN between all three tables.


In [ ]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create tables
cur.execute('CREATE TABLE Orders (order_id INT, order_date TEXT, customer_id INT)')
cur.execute('CREATE TABLE Customers (customer_id INT, customer_name TEXT)')
cur.execute('CREATE TABLE Order_Details (order_id INT, product_id INT, quantity INT)')

# Insert data
cur.executemany('INSERT INTO Orders VALUES (?, ?, ?)', [
    (1, '2024-01-02', 1),
    (2, '2024-01-05', 2)
])

cur.executemany('INSERT INTO Customers VALUES (?, ?)', [
    (1, 'Alice'),
    (2, 'Bob')
])

cur.executemany('INSERT INTO Order_Details VALUES (?, ?, ?)', [
    (1, 101, 2),
    (1, 102, 1),
    (2, 101, 3)
])

# Query: order_id, customer_name, total quantity
cur.execute('''
SELECT o.order_id, c.customer_name, SUM(od.quantity) AS total_quantity
FROM Orders o
INNER JOIN Customers c ON o.customer_id = c.customer_id
INNER JOIN Order_Details od ON o.order_id = od.order_id
GROUP BY o.order_id, c.customer_name
''')

# Fetch and display results
for row in cur.fetchall():
    print(row)

conn.close()


(1, 'Alice', 3)
(2, 'Bob', 3)


## **SQL Commands**

### Q1. Identify the primary keys and foreign keys in Maven Movies DB. Discuss the differences

**Primary Key (PK):**  
- A primary key is a column (or combination of columns) that uniquely identifies each row in a table.  
- It **cannot contain NULL values** and must be unique.  
- Examples:  
  - `actor_id` in the `actors` table  
  - `customer_id` in the `customers` table  
  - `film_id` in the `films` table  

**Foreign Key (FK):**  
- A foreign key is a column (or combination of columns) that **creates a relationship between two tables**.  
- It references the primary key of another table to enforce referential integrity.  
- Examples:  
  - `customer_id` in the `rentals` table references `customer_id` in `customers`  
  - `film_id` in the `inventory` table references `film_id` in `films`  
  - `actor_id` in the `film_actor` table references `actor_id` in `actors`  

**Differences:**

| Feature              | Primary Key                       | Foreign Key                                 |
|----------------------|----------------------------------|--------------------------------------------|
| Purpose              | Uniquely identifies a row        | Establishes a relationship between tables  |
| Uniqueness           | Must be unique                   | Can have duplicates in the child table     |
| NULL Values           | Not allowed                      | Can be NULL (optional)                      |
| Table Dependency      | Defined within the table itself  | References another table                    |
| Integrity Enforcement | Ensures entity integrity          | Ensures referential integrity               |

**Summary:**  
- Primary keys identify rows uniquely in a table.  
- Foreign keys link tables and maintain consistency between them.

### Q2. List all details of actors

**Answer (SQL Query):**
```sql
SELECT *
FROM actors;


### Q3. List all customer information from the database

**Answer (SQL Query):**
```sql
SELECT *
FROM customers;


### Q4. List different countries

**Answer (SQL Query):**
```sql
SELECT DISTINCT country_name
FROM countries;


### Q5. Display all active customers

**Answer (SQL Query):**
```sql
SELECT *
FROM customers
WHERE active = 1;


### Q6. List of all rental IDs for customer with ID 1

**Answer (SQL Query):**
```sql
SELECT rental_id
FROM rentals
WHERE customer_id = 1;


### Q7. Display all the films whose rental duration is greater than 5

**Answer (SQL Query):**
```sql
SELECT *
FROM films
WHERE rental_duration > 5;


### Q8. List the total number of films whose replacement cost is greater than $15 and less than $20

**Answer (SQL Query):**
```sql
SELECT COUNT(*) AS total_films
FROM films
WHERE replacement_cost > 15 AND replacement_cost < 20;


### Q9. Display the count of unique first names of actors

**Answer (SQL Query):**
```sql
SELECT COUNT(DISTINCT first_name) AS unique_first_names
FROM actors;


### Q10. Display the first 10 records from the customer table

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
LIMIT 10;


### Q11. Display the first 3 records from the customer table whose first name starts with 'b'

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE 'b%'
LIMIT 3;


### Q12. Display the names of the first 5 movies which are rated as 'G'

**Answer (SQL Query):**
```sql
SELECT film_title
FROM films
WHERE rating = 'G'
LIMIT 5;


### Q13. Find all customers whose first name starts with "a"

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE 'a%';


### Q14. Find all customers whose first name ends with "a"

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE '%a';


### Q15. Display the list of first 4 cities which start and end with 'a'

**Answer (SQL Query):**
```sql
SELECT city_name
FROM city
WHERE city_name LIKE 'a%a'
LIMIT 4;


### Q16. Find all customers whose first name have "NI" in any position

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE '%NI%';


### Q17. Find all customers whose first name have "r" in the second position

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE '_r%';


### Q18. Find all customers whose first name starts with "a" and are at least 5 characters in length

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE 'a%'
  AND LENGTH(first_name) >= 5;


### Q19. Find all customers whose first name starts with "a" and ends with "o"

**Answer (SQL Query):**
```sql
SELECT *
FROM customer
WHERE first_name LIKE 'a%o';


### Q20. Get the films with PG and PG-13 rating using IN operator

**Answer (SQL Query):**
```sql
SELECT *
FROM films
WHERE rating IN ('PG', 'PG-13');


### Q21. Get the films with length between 50 to 100 using BETWEEN operator

**Answer (SQL Query):**
```sql
SELECT *
FROM films
WHERE length BETWEEN 50 AND 100;


### Q22. Get the top 50 actors using LIMIT operator

**Answer (SQL Query):**
```sql
SELECT *
FROM actors
LIMIT 50;


### Q23. Get the distinct film IDs from inventory table

**Answer (SQL Query):**
```sql
SELECT DISTINCT film_id
FROM inventory;


## **Functions Basic Aggregate Functions:**

### Q1. Retrieve the total number of rentals made in the Sakila database

**Answer (SQL Query):**
```sql
SELECT COUNT(*) AS total_rentals
FROM rental;


### Q2. Find the average rental duration (in days) of movies rented from the Sakila database

**Answer (SQL Query):**
```sql
SELECT AVG(rental_duration) AS avg_rental_duration
FROM film;


### Q3. Display the first name and last name of customers in uppercase

**Answer (SQL Query):**
```sql
SELECT UPPER(first_name) AS first_name_upper,
       UPPER(last_name) AS last_name_upper
FROM customer;


### Q4. Extract the month from the rental date and display it alongside the rental ID

**Answer (SQL Query):**
```sql
SELECT rental_id,
       MONTH(rental_date) AS rental_month
FROM rental;


### Q5. Retrieve the count of rentals for each customer (display customer ID and the count of rentals)

**Answer (SQL Query):**
```sql
SELECT customer_id,
       COUNT(*) AS rental_count
FROM rental
GROUP BY customer_id;


### Q6. Find the total revenue generated by each store

**Answer (SQL Query):**
```sql
SELECT store_id,
       SUM(amount) AS total_revenue
FROM payment
GROUP BY store_id;


### Q7. Determine the total number of rentals for each category of movies

**Answer (SQL Query):**
```sql
SELECT fc.category_id,
       c.name AS category_name,
       COUNT(r.rental_id) AS total_rentals
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
GROUP BY fc.category_id, c.name;


### Q8. Find the average rental rate of movies in each language

**Answer (SQL Query):**
```sql
SELECT l.language_id,
       l.name AS language_name,
       AVG(f.rental_rate) AS avg_rental_rate
FROM film f
JOIN language l ON f.language_id = l.language_id
GROUP BY l.language_id, l.name;


## **Joins**

### Q9. Display the title of the movie, customer's first name, and last name who rented it

**Answer (SQL Query):**
```sql
SELECT f.title,
       c.first_name,
       c.last_name
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN customer c ON r.customer_id = c.customer_id;


### Q10. Retrieve the names of all actors who have appeared in the film "Gone with the Wind"

**Answer (SQL Query):**
```sql
SELECT a.first_name,
       a.last_name
FROM film_actor fa
JOIN actor a ON fa.actor_id = a.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.title = 'Gone with the Wind';


### Q11. Retrieve the customer names along with the total amount they've spent on rentals

**Answer (SQL Query):**
```sql
SELECT c.first_name,
       c.last_name,
       SUM(p.amount) AS total_spent
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
JOIN rental r ON p.rental_id = r.rental_id
GROUP BY c.customer_id, c.first_name, c.last_name;


### Q12. List the titles of movies rented by each customer in a particular city (e.g., 'London')

**Answer (SQL Query):**
```sql
SELECT c.first_name,
       c.last_name,
       GROUP_CONCAT(f.title) AS rented_movies
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE ci.city = 'London'
GROUP BY c.customer_id, c.first_name, c.last_name;


## **Advanced Joins and GROUP BY:**

### Q13. Display the top 5 rented movies along with the number of times they've been rented

**Answer (SQL Query):**
```sql
SELECT f.title,
       COUNT(r.rental_id) AS rental_count
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id, f.title
ORDER BY rental_count DESC
LIMIT 5;


### Q14. Determine the customers who have rented movies from both stores (store ID 1 and store ID 2)

**Answer (SQL Query):**
```sql
SELECT c.customer_id,
       c.first_name,
       c.last_name
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
WHERE i.store_id IN (1, 2)
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING COUNT(DISTINCT i.store_id) = 2;


## **Windows Function:**

### Q1. Rank the customers based on the total amount they've spent on rentals

**Answer (SQL Query):**
```sql
SELECT c.customer_id,
       c.first_name,
       c.last_name,
       SUM(p.amount) AS total_spent,
       RANK() OVER (ORDER BY SUM(p.amount) DESC) AS rank_customer
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name;


### Q2. Calculate the cumulative revenue generated by each film over time

**Answer (SQL Query):**
```sql
SELECT f.film_id,
       f.title,
       p.payment_date,
       SUM(p.amount) OVER (
           PARTITION BY f.film_id
           ORDER BY p.payment_date
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS cumulative_revenue
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN payment p ON r.rental_id = p.rental_id;


### Q3. Determine the average rental duration for each film, considering films with similar lengths

**Answer (SQL Query):**
```sql
SELECT f.film_id,
       f.title,
       f.length,
       f.rental_duration,
       AVG(f.rental_duration) OVER (PARTITION BY f.length) AS avg_rental_duration_by_length
FROM film f;


### Q4. Identify the top 3 films in each category based on their rental counts

**Answer (SQL Query):**
```sql
SELECT c.name AS category_name,
       f.title,
       COUNT(r.rental_id) AS rental_count,
       RANK() OVER (PARTITION BY c.category_id ORDER BY COUNT(r.rental_id) DESC) AS film_rank
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
GROUP BY c.category_id, c.name, f.title
QUALIFY RANK() OVER (PARTITION BY c.category_id ORDER BY COUNT(r.rental_id) DESC) <= 3;


### Q5. Calculate the difference in rental counts between each customer's total rentals and the average rentals across all customers

**Answer (SQL Query):**
```sql
SELECT c.customer_id,
       c.first_name,
       c.last_name,
       COUNT(r.rental_id) AS total_rentals,
       COUNT(r.rental_id) - AVG(COUNT(r.rental_id)) OVER () AS diff_from_avg
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name;


### Q6. Find the monthly revenue trend for the entire rental store over time

**Answer (SQL Query):**
```sql
SELECT DATE_FORMAT(payment_date, '%Y-%m') AS month,
       SUM(amount) AS monthly_revenue,
       SUM(SUM(amount)) OVER (ORDER BY DATE_FORMAT(payment_date, '%Y-%m')) AS cumulative_revenue
FROM payment
GROUP BY DATE_FORMAT(payment_date, '%Y-%m')
ORDER BY month;


### Q7. Identify the customers whose total spending on rentals falls within the top 20% of all customers

**Answer (SQL Query):**
```sql
WITH customer_spending AS (
    SELECT c.customer_id,
           c.first_name,
           c.last_name,
           SUM(p.amount) AS total_spent
    FROM customer c
    JOIN payment p ON c.customer_id = p.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
),
ranked_customers AS (
    SELECT customer_id,
           first_name,
           last_name,
           total_spent,
           PERCENT_RANK() OVER (ORDER BY total_spent DESC) AS spending_percentile
    FROM customer_spending
)
SELECT customer_id,
       first_name,
       last_name,
       total_spent
FROM ranked_customers
WHERE spending_percentile <= 0.20
ORDER BY total_spent DESC;


### Q8. Calculate the running total of rentals per category, ordered by rental count

**Answer (SQL Query):**
```sql
WITH category_rentals AS (
    SELECT c.name AS category_name,
           COUNT(r.rental_id) AS rental_count
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    GROUP BY c.name
)
SELECT category_name,
       rental_count,
       SUM(rental_count) OVER (ORDER BY rental_count DESC) AS running_total
FROM category_rentals
ORDER BY rental_count DESC;


### Q9. Find the films that have been rented less than the average rental count for their respective categories

**Answer (SQL Query):**
```sql
WITH film_rentals AS (
    SELECT f.film_id,
           f.title,
           c.name AS category_name,
           COUNT(r.rental_id) AS rental_count
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    GROUP BY f.film_id, f.title, c.name
),
category_avg AS (
    SELECT category_name,
           AVG(rental_count) AS avg_rental_count
    FROM film_rentals
    GROUP BY category_name
)
SELECT fr.film_id,
       fr.title,
       fr.category_name,
       fr.rental_count,
       ca.avg_rental_count
FROM film_rentals fr
JOIN category_avg ca
  ON fr.category_name = ca.category_name
WHERE fr.rental_count < ca.avg_rental_count
ORDER BY fr.category_name, fr.rental_count;


### Q10. Identify the top 5 months with the highest revenue and display the revenue generated in each month

**Answer (SQL Query):**
```sql
SELECT strftime('%Y-%m', payment_date) AS month,
       SUM(amount) AS total_revenue
FROM payment
GROUP BY strftime('%Y-%m', payment_date)
ORDER BY total_revenue DESC
LIMIT 5;


## **Normalisation & CTE**

### Q1. First Normal Form (1NF):
**a. Identify a table in the Sakila database that violates 1NF. Explain how you would normalize it to achieve 1NF.**

**Answer:**

- **Table that may violate 1NF:** `film_actor` or any hypothetical table where multiple values are stored in a single column.  
  Example: Suppose we have a table `customer_rentals`:

| customer_id | rental_ids      |
|-------------|----------------|
| 1           | 101, 102, 103  |
| 2           | 104, 105       |

- **Problem:** The `rental_ids` column contains multiple values in a single cell, violating 1NF.

- **Normalization to achieve 1NF:** Split the multiple values into separate rows so that each column contains only atomic (single) values:

| customer_id | rental_id |
|-------------|-----------|
| 1           | 101       |
| 1           | 102       |
| 1           | 103       |
| 2           | 104       |
| 2           | 105       |

- **Result:** Each column now contains atomic values, eliminating repeating groups and satisfying 1NF.


### Q2. Second Normal Form (2NF)
**a. Choose a table in Sakila and describe how you would determine whether it is in 2NF. If it violates 2NF, explain the steps to normalize it.**

**Answer:**

- **Example Table:** `payment` (assuming a composite primary key of `customer_id` and `rental_id`).

| customer_id | rental_id | amount | customer_name | rental_date |
|-------------|-----------|--------|---------------|-------------|

- **Check for 2NF:**
  - A table is in 2NF if it is already in 1NF **and** all non-key columns are fully functionally dependent on the entire primary key.
  - **Problem:** `customer_name` depends only on `customer_id`, not the combination of `customer_id` and `rental_id`.  
    This violates 2NF because of partial dependency.

- **Normalization to achieve 2NF:**
  1. Separate the table into two tables:
     - `payment` table: keeps columns fully dependent on the composite key (`customer_id`, `rental_id`, `amount`, `rental_date`).
     - `customer` table: keeps `customer_id` and `customer_name`.
     
- **Result:**  
  - `payment` table:

| customer_id | rental_id | amount | rental_date |
|-------------|-----------|--------|-------------|

  - `customer` table:

| customer_id | customer_name |
|-------------|---------------|

- **Benefit:** Eliminates partial dependencies and ensures 2NF.


### Q3. Third Normal Form (3NF)
**a. Identify a table in Sakila that violates 3NF. Describe the transitive dependencies present and outline the steps to normalize the table to 3NF.**

**Answer:**

- **Example Table:** `rental`  

| rental_id | inventory_id | customer_id | store_id | store_address | rental_date |
|-----------|-------------|-------------|---------|---------------|-------------|

- **Check for 3NF:**
  - A table is in 3NF if it is in 2NF **and** all non-key columns are **not transitively dependent** on the primary key.
  - **Problem:** `store_address` depends on `store_id`, which depends on the primary key `rental_id`.  
    This is a transitive dependency: `rental_id → store_id → store_address`.

- **Normalization to achieve 3NF:**
  1. Remove transitive dependencies by creating separate tables:
     - `rental` table: keeps columns fully dependent on the primary key (`rental_id`, `inventory_id`, `customer_id`, `store_id`, `rental_date`).
     - `store` table: keeps `store_id` and `store_address`.
     
- **Result:**  
  - `rental` table:

| rental_id | inventory_id | customer_id | store_id | rental_date |
|-----------|-------------|-------------|---------|-------------|

  - `store` table:

| store_id | store_address |
|----------|---------------|

- **Benefit:** Eliminates transitive dependencies and ensures 3NF.


### Q4. Normalization Process
**a. Take a specific table in Sakila and guide through the process of normalizing it from the initial unnormalized form up to at least 2NF.**

**Answer:**

- **Example Table (Unnormalized Form - UNF):** `customer_rentals`  

| customer_id | customer_name | rental1_movie | rental1_date | rental2_movie | rental2_date |
|-------------|---------------|---------------|--------------|---------------|--------------|

- **Step 1: Convert to First Normal Form (1NF)**  
  - Eliminate repeating groups and ensure each column contains atomic values.
  
| customer_id | customer_name | movie_title | rental_date |
|-------------|---------------|-------------|-------------|
| 1           | Alice         | Movie A     | 2024-01-01  |
| 1           | Alice         | Movie B     | 2024-01-03  |
| 2           | Bob           | Movie C     | 2024-01-02  |

- **Step 2: Convert to Second Normal Form (2NF)**  
  - Ensure table is in 1NF and all non-key attributes are fully functionally dependent on the **entire primary key**.
  - Here, primary key is a **composite key**: (`customer_id`, `movie_title`, `rental_date`).  
  - Move partial dependencies (like `customer_name`) to a separate `customer` table.

- **Resulting Tables in 2NF:**  

1. `customer` table:

| customer_id | customer_name |
|-------------|---------------|
| 1           | Alice         |
| 2           | Bob           |

2. `rental` table:

| customer_id | movie_title | rental_date |
|-------------|-------------|-------------|
| 1           | Movie A     | 2024-01-01  |
| 1           | Movie B     | 2024-01-03  |
| 2           | Movie C     | 2024-01-02  |

- **Benefit:**  
  - Eliminates partial dependencies.
  - Reduces redundancy and ensures data consistency.


### Q5. Write a query using a CTE to retrieve the distinct list of actor names and the number of films they have acted in from the actor and film_actor tables

**Answer (SQL Query using CTE):**
```sql
WITH actor_film_count AS (
    SELECT a.actor_id,
           a.first_name,
           a.last_name,
           COUNT(fa.film_id) AS film_count
    FROM actor a
    JOIN film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY a.actor_id, a.first_name, a.last_name
)
SELECT first_name,
       last_name,
       film_count
FROM actor_film_count
ORDER BY film_count DESC;


### Q6. Create a CTE that combines information from the film and language tables to display the film title, language name, and rental rate

**Answer (SQL Query using CTE):**
```sql
WITH film_language_info AS (
    SELECT f.film_id,
           f.title,
           l.name AS language_name,
           f.rental_rate
    FROM film f
    JOIN language l ON f.language_id = l.language_id
)
SELECT title,
       language_name,
       rental_rate
FROM film_language_info
ORDER BY title;


### Q7. Write a query using a CTE to find the total revenue generated by each customer (sum of payments) from the customer and payment tables

**Answer (SQL Query using CTE):**
```sql
WITH customer_revenue AS (
    SELECT c.customer_id,
           c.first_name,
           c.last_name,
           SUM(p.amount) AS total_revenue
    FROM customer c
    JOIN payment p ON c.customer_id = p.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)
SELECT customer_id,
       first_name,
       last_name,
       total_revenue
FROM customer_revenue
ORDER BY total_revenue DESC;


### Q8. Utilize a CTE with a window function to rank films based on their rental duration from the film table

**Answer (SQL Query using CTE and Window Function):**
```sql
WITH film_ranking AS (
    SELECT film_id,
           title,
           rental_duration,
           RANK() OVER (ORDER BY rental_duration DESC) AS duration_rank
    FROM film
)
SELECT film_id,
       title,
       rental_duration,
       duration_rank
FROM film_ranking
ORDER BY duration_rank;


### Q9. Create a CTE to list customers who have made more than two rentals, and then join this CTE with the customer table to retrieve additional customer details

**Answer (SQL Query using CTE and Join):**
```sql
WITH frequent_customers AS (
    SELECT customer_id,
           COUNT(rental_id) AS rental_count
    FROM rental
    GROUP BY customer_id
    HAVING COUNT(rental_id) > 2
)
SELECT c.customer_id,
       c.first_name,
       c.last_name,
       fc.rental_count
FROM frequent_customers fc
JOIN customer c ON fc.customer_id = c.customer_id;


### Q10. Write a query using a CTE to find the total number of rentals made each month, considering the rental_date from the rental table

**Answer (SQL Query using CTE for Date Calculations):**
```sql
WITH monthly_rentals AS (
    SELECT strftime('%Y-%m', rental_date) AS rental_month,
           COUNT(rental_id) AS total_rentals
    FROM rental
    GROUP BY rental_month
)
SELECT rental_month,
       total_rentals
FROM monthly_rentals
ORDER BY rental_month;


### Q11. Create a CTE to generate a report showing pairs of actors who have appeared in the same film together, using the film_actor table

**Answer (SQL Query using CTE and Self-Join):**
```sql
WITH actor_pairs AS (
    SELECT fa1.film_id,
           fa1.actor_id AS actor1_id,
           fa2.actor_id AS actor2_id
    FROM film_actor fa1
    JOIN film_actor fa2
      ON fa1.film_id = fa2.film_id
     AND fa1.actor_id < fa2.actor_id
)
SELECT ap.film_id,
       a1.first_name || ' ' || a1.last_name AS actor1,
       a2.first_name || ' ' || a2.last_name AS actor2
FROM actor_pairs ap
JOIN actor a1 ON ap.actor1_id = a1.actor_id
JOIN actor a2 ON ap.actor2_id = a2.actor_id
ORDER BY ap.film_id;


### Q12. Implement a recursive CTE to find all employees in the staff table who report to a specific manager, considering the reports_to column

**Answer (SQL Query using Recursive CTE):**
```sql
WITH RECURSIVE employee_hierarchy AS (
    -- Anchor member: select the manager
    SELECT staff_id,
           first_name || ' ' || last_name AS employee_name,
           reports_to
    FROM staff
    WHERE staff_id = 2  -- Replace with the manager's ID

    UNION ALL

    -- Recursive member: find employees reporting to the above
    SELECT s.staff_id,
           s.first_name || ' ' || s.last_name AS employee_name,
           s.reports_to
    FROM staff s
    JOIN employee_hierarchy eh ON s.reports_to = eh.staff_id
)
SELECT *
FROM employee_hierarchy;
